## [Previous Section](./ClassesDecorator.ipynb)

## A Metaclass Solution for `Checked`

In [1]:
from collections.abc import Callable  
from typing import Any, NoReturn, get_type_hints

In [2]:
class Field:
    
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.storage_name = '_' + name
        self.constructor = constructor

    def __get__(self, instance, owner=None):
        # If `__get__` gets `None` as the `instance` argument, the descriptor is being read from
        # the managed class itself, not a managed instance. So we return the decriptor. 
        if isinstance is None:
            return self
        # Otherwise, return the value stored in the attribute named `storage_name`. 
        return getattr(instance, self.storage_name)
    
    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not callable with {self.name}: {type_name}'
                raise TypeError(msg) from e
        setattr(instance, self.storage_name, value)

In [3]:
class CheckedMeta(type):

    def __new__(meta_cls, cls_name, bases, cls_dict):
        if '__slots__' not in cls_dict:
            slots = []
            # To get the type hints in prior examples, we used `typing.get_type_hints`, 
            # but that requires an existing class as the first argument. At this point, 
            # the class we are configuring does not exist yet, so we need to retrieve the
            # `__annotations__` directly from the `cls_dict` —— the namespace of the class under construction, 
            # which Python passes as the last argument to the metaclass `__new__`
            type_hints = cls_dict.get('__annotations__', {})
            for name, constructor in type_hints.items():
                field = Field(name, constructor)
                cls_dict[name] = field
                slots.append(field.storage_name)
            # populate the `__slots__` entry in `cls_dict` —— the namespace of the class under construction. 
            cls_dict['__slots__'] = slots

        # Finally, we call `super().__new__`
        return super().__new__(
            meta_cls, cls_name, bases, cls_dict)

The last part of this, is the `Checked` base class. 

The code for this version of `Checked` is the same as `Checked` in the *previous section*, with three changes:
1. Added an empty `__slots__` to signal to `CheckedMeta.__new__` that this class doesn't require special processing. 
2. Removed `__setattr__`. Its job is now done by `CheckedMeta.__new__`. 
3. Removed `__setattr__`. It became redundant because adding `__slots__` to the user-defined class prevents setting undeclared attributes. 

In [4]:
class Checked(metaclass=CheckedMeta):

    __slots__ = ()  # skip `CheckedMeta.__new__` processing

    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unknown_attrs(*kwargs)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no Attribute{plural} {extra}')
    
    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }
    
    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In [5]:
class Movie(Checked):
    title: str
    year: int
    box_office: float

In [6]:
movie = Movie(title='The Godfather', year=1972)
print(movie)
print(movie.title)

Movie(title='The Godfather', year=1972, box_office=0.0)
The Godfather


## A Metaclass Hack with `__prepare__`

In [7]:
class WilyDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__next_value = 0

    def __missing__(self, key):
        if key.startswith('__') and key.endswith('__'):
            raise KeyError(key)
        self[key] = value = self.__next_value
        self.__next_value += 1
        return value

In [8]:
class AutoConstMeta(type):
    def __prepare__(name, bases, **kwargs):
        return WilyDict()


class AutoConst(metaclass=AutoConstMeta):
    pass

In [9]:
class Flavor(AutoConst):
    banana
    coconut
    vanilla

print('Flavor.vanilla ==', Flavor.vanilla)

Flavor.vanilla == 2


When Python processes the namespace of the user's class and reads `banana`, it looks up that name in the mapping provided by `__prepare__`: an instance of `WilyDict`. `WilyDict` implements `__missing__`. The `WilyDict` instance initially has no `banana` key, so the `__missing__` method returning that value. Python is happy with that, then tries retrieve `coconut`. `WilyDict` promptly adds that entry with value `1`, returning it. The same happens with `vanilla`, which is then mapped to `2`. 

## Wrapping Up

Metaclasses, as well as class decorators and `__init_subclass__` are useful for:
- Subclass registration
- Subclass structural validation
- Applying decorators to many methods at once
- Object serialization
- Object-relational mapping
- Object-based persistence
- Implementing special methods at the class level
- Implementing class features found in other languages, such as [traits](https://fpy.li/24-17) and [aspect-oriented programming](https://fpy.li/24-18). 

In [10]:
class PluginBase:
    plugins = []

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        cls.plugins.append(cls)

class MyPlugin(PluginBase):
    pass

class AnotherPlugin(PluginBase):
    pass


print(PluginBase.plugins)  

[<class '__main__.MyPlugin'>, <class '__main__.AnotherPlugin'>]
